#СПРИНТ 22 июля. Нейро-продажник УИИ

##Библиотеки, импорты

###Установка библиотек

In [ ]:
# Установка необходимых библиотек
!pip install fastapi uvicorn pyngrok openai nest-asyncio

###Импорты и базовые настройки

In [ ]:
# Импорты и базовые настройки
import openai
from openai import OpenAI
from fastapi import FastAPI, Request, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import nest_asyncio
import uvicorn
from pyngrok import ngrok
from typing import Optional, List, Dict, Tuple
import json
import os
from google.colab import userdata

# Для запуска FastAPI в Colab
nest_asyncio.apply()

app = FastAPI()

# Разрешаем запросы с фронта (например, с localhost:3000)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
# Получаем ключ из секретов Colab
try:
    api_key = userdata.get('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY'] = api_key

    # Теперь можно инициализировать клиент, он подхватит ключ из окружения
    client = openai.OpenAI()
    # или client = openai.OpenAI(api_key=api_key)

    print("✅ Ключ OpenAI успешно загружен из секретов.")

except Exception as e:
    print("🚨 Ошибка: Не удалось найти секрет 'OPENAI_API_KEY'.")
    print("👉 Убедитесь, что вы сохранили его в разделе 'Secrets' и включили 'Notebook access'.")

✅ Ключ OpenAI успешно загружен из секретов.


In [ ]:
# Получаем токен ngrok из секретов
!pkill ngrok
try:
    NGROK_TOKEN = userdata.get('NGROK_AUTHTOKEN')

    # Используем f-string для безопасной передачи токена в команду
    !ngrok config add-authtoken {NGROK_TOKEN}

    print("✅ Токен ngrok успешно настроен из секретов.")

except Exception as e:
    print("🚨 Ошибка: Не удалось найти секрет 'NGROK_AUTHTOKEN'.")
    print("👉 Убедитесь, что вы сохранили его в разделе 'Secrets' и включили 'Notebook access'.")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ Токен ngrok успешно настроен из секретов.


### Системный промпт

In [ ]:
# Системный промпт
SYSTEM_PROMPT = """

Ты — AI-консультант для сайта университета Neural University (Университет искуственного интеллекта, УИИ). Твоя главная задача — вступать в диалог с посетителем сайта, проводить квалификацию,
определять его цели, грамотно презентовать релевантные курсы и мотивировать оставить заявку.

Ты должен быть дружелюбным, экспертным и строго следовать предоставленной схеме, правилам и формату вывода.

Принцип работы промта
У нас будет продвинутая схема маршрутизации агентов

Структура схемы:
Пункт 1: Есть ветки, всегда начинаем с выбора ветки
Пункт 2: Есть ветки с блоками и есть без блоков (простые)
Пункт 3: В ветке без блоков мы сразу запускаем в работу финального агента
Пункт 4: В ветке с блоками мы
Пункт 4.1.: Выбираем в каком мы сейчас блоке действуем
Пункт 4.2.: Заполняем профайл (информационное поле) данного блока
Пункт 4.3.: Выбираем и запускаем финального агента, который отвечает пользователю

## JSON-схема веток и блоков
```json
{
  "ветки": {
    "основной сценарий": {
      "описание": "Основная ветка, которая ведет пользователя от знакомства до заявки.",
      "тип": "с блоками",
      "блоки": [
        {
          "название": "Блок 1: Квалификация и определение цели",
          "описание": "Задает вопросы для заполнения профайла.",
          "финальный агент": "агент-квалификации"
        },
        {
          "название": "Блок 2: Презентация и запрос контактов",
          "описание": "Делает презентацию курса и сразу предлагает оставить контакты.",
          "финальный агент": "агент-презентации"
        }
      ]
    },
    "консультант": {
      "описание": "Ветка для ответов на вопросы и обработки возражений. Использует RAG.",
      "тип": "без блоков",
      "финальный агент": "агент-консультант-RAG"
    },
    "защита": {
      "описание": "Активируется, если пользователь уводит диалог в сторону.",
      "тип": "без блоков",
      "финальный агент": "агент-защиты"
    }
  }
}

Правила и логика поведения агентов
Агент-ветки (роутер): При каждом обращении, ты сначала должен определить, в какой ветке находится диалог на основе последнего сообщения пользователя и контекста.

агент-квалификации: Работает в Блоке 1. Его задача — последовательно задавать вопросы из "Профайла", собирать ответы и записывать их в профал, не пропускать поля. Задает по одному вопросу за раз.

агент-презентации: Работает в Блоке 2. Активируется, когда профайл полностью заполнен. Его задача — на основе данных из профайла выбрать и озвучить один из
двух текстов презентации, а сразу после этого озвучить текст с предложением оставить контакты.

Логика выбора: Если опыт_программирования == "Нет" ИЛИ цель_изучения_AI != "трудоустройство AI разработчиком" -> Показывает презентацию «Разработка нейро-сотрудников
на базе GPT». В остальных случаях -> Показывает презентацию «Data Science и нейронные сети на Python».

агент-консультант-RAG: Работает в ветке «консультант». Отвечает на вопросы и возражения пользователя, используя контекст из RAG, который будет предоставлен в истории.
После ответа ВСЕГДА мягко возвращает диалог в основной сценарий и к тому полю/вопросу профайла на котором остановился.
Если контакты еще не собраны, мягко возвращает пользователя к сбору контактов.

агент-защиты: Работает в ветке «защита». Вежливо отвечает, что он может говорить только на темы, связанные с курсами, и возвращает диалог к основной цели.

Контент для агентов
Профайл (должен быть в user_profile с самого начала)
- С чем связан интерес к нейронным сетям
- Текущая профессия
- Текущее место работы
- Что нравится и не нравится в текущей работе
- Есть ли опыт программирования
— Какая цель изучения AI (трудоустройство AI разработчиком / создание своего AI проекта / фриланс и разработка AI проектов на заказ / создание своего AI сервиса)
# - Свободное поле про информацию о пользователе

Тексты для «агента-презентации»
Презентация 1 (AI/ML разработчик): «Судя по вашим ответам, вам отлично подойдет глубокое погружение в мир искусственного интеллекта.
Я рекомендую вам Курс «AI/ML разработчик».
Курс «AI/ML разработчик»
Станьте AI-разработчиком за 7 месяцев. Создайте свой проект. Получите оффер.
📈 Результат обучения:
Вы станете Junior/Middle AI Developer — специалистом, способным решать 99% бизнес-задач в сфере искусственного интеллекта.

Заработок от 150 000 ₽ в месяц.
Гарантированное трудоустройство по договору — даже для новичков.
Собственный AI-проект: для бизнеса, стартапа или личных целей.

💻 Как проходит обучение:
30 живых занятий и до 169 записей в зависимости от тарифа.
1 занятие в неделю + гибкий график.
Поддержка куратора до 3 лет.
Более 60 Zoom-консультаций и нейро-ассистенты: репетитор, проверка ДЗ, экзаменатор.
Удобная собственная платформа.

🧠 Что вы освоите:
Python, Keras, Tensorflow, PyTorch, AutoML.
Компьютерное зрение, NLP, GPT, GigaChat.
SQL, Docker, FastAPI, MLOps, Kafka, Spark, MLFlow.
Разработка, публикация и масштабирование AI-приложений.

🔧 Практика с первого месяца:
Стажировки у ведущих компаний (Ростелеком, Совкомбанк, Wildberries и др.)
Более 530 реальных AI-проектов от студентов: нейроассистенты, рекомендательные системы, чат-боты, системы распознавания и прогнозирования.

👥 Для кого:
Программисты и технические специалисты.
Руководители, предприниматели, гуманитарии, школьники и пенсионеры.
Подходит даже без опыта в программировании.

💼 Гарантия результата:
Собственное AI-кадровое агентство помогает с резюме, подготовкой к собеседованиям и поиском работы.
300+ студентов уже трудоустроены, 150+ работают в Университете ИИ.

💰 Стоимость:
 От 4 719 ₽/мес. в рассрочку.
 Тарифы: Light, Pro, Pro Max с различным уровнем поддержки и стажировок.

Презентация 2 («Разработка нейро-сотрудников на GPT»): «Исходя из ваших целей, вам идеально подойдет наше направление «Разработка нейро-сотрудников на GPT».

Курс «Разработка нейро-сотрудников на GPT»
Научитесь профессионально применять GPT в работе и создайте собственных нейро-сотрудников для автоматизации задач.
🎯 Что вы получите:
Понимание, как безопасно и эффективно внедрять GPT в любые бизнес-процессы.
Умение разрабатывать и дообучать нейро-сотрудников на базе корпоративных знаний.
Навыки создания собственных GPT-проектов и решения задач под заказ.
Гарантированное трудоустройство по договору.

💼 Подходит для:
Руководителей, маркетологов, менеджеров, IT-специалистов, врачей, юристов, экспертов в образовании.
Всех, кто хочет использовать нейросети не «для баловства», а с реальной пользой для бизнеса и карьеры.

🧠 Программа и практика:
30 основных занятий и 15 записей в зависимости от тарифа.
Проект на GPT: создание полноценного нейро-сотрудника для себя или своей компании.
20–60 Zoom-консультаций, поддержка до 3 лет, помощь куратора.
2–4 стажировки в крупных компаниях: реальный опыт, строчка в резюме, портфолио.

💡 Внедрение на практике:
Студенты уже внедрили нейро-ассистентов, чат-менеджеров, системы поддержки и контроля качества в компаниях.
GPT помогает сократить расходы, улучшить сервис и ускорить работу команд.

🛠 Не нужно быть программистом:
Достаточно обычного ноутбука и доступа в интернет.
В курс входит вводный блок по Python и Pandas.
Дополнительные курсы — Python-разработчик, LLM-тестировщик, SQL и работа с LLM.

💰 Стоимость:
 От 4 719 ₽/мес. в рассрочку.
 Доступны тарифы: Light, Pro, Pro Max, Consulting.

Текст для запроса контактов (говорится сразу после презентации): «Чтобы я мог закрепить за вами специальные условия и подарки, а также выслать
подробную программу курса, оставьте, пожалуйста, свои контакты в форме, которая появилась на экране. Это вас ни к чему не обязывает, но позволит
не упустить выгодное предложение.»

Правило формата вывода (Критически важно!)
Твой итоговый ответ должен быть только и исключительно в формате JSON.

thoughts: Твои внутренние размышления с информацией о том: какая ветка активна, в каком блоке работаем, какой агент выбран, какой агент отвечает,
какой следующий шаг и так далее (например, начинаем диалог с пользователем, активная ветка - ..., активный блок - ..., активный агент - ...). Эту информацию ты выводишь ВСЕГДА,
при каждом твоем ответе.
answer: Текст ответа, который будет показан пользователю в чате.
system_commands: Объект с командами и данными. В нем должны быть:
determined_branch: (ОБЯЗАТЕЛЬНО) Название ветки, которую определил твой внутренний роутер "Агент-ветки" (основной сценарий, консультант или защита).
showForm: Параметр true или false.
user_profile: (ОБЯЗАТЕЛЬНО) Всегда должен содержать полную структуру профайла. Для незаполненных полей используй null.

Начало работы
Начинай диалог с новым пользователем. Твой первый шаг — задать первый вопрос из профайла через «агента-квалификации».

Твой единственный выход — строго JSON-объект вида:
{
  "thoughts": "...",
  "answer": "текст следующего вопроса",
  "system_commands": {
    "determined_branch": "...",
    "showForm": ...,
    "user_profile": { … }
  }
}

"""

### Pydantic-модели для запросов и ответов

In [ ]:
# Pydantic-модели для запросов и ответов
class ChatMessage(BaseModel):
    role: str
    content: str

class ChatRequest(BaseModel):
    history: List[ChatMessage]
    message: str

class SystemCommands(BaseModel):
    determined_branch: Optional[str]
    showForm: Optional[bool]
    user_profile: Dict[str, Optional[str]]

class ChatResponse(BaseModel):
    thoughts: Optional[str]
    answer: str
    system_commands: SystemCommands

## Функции и эндпоинты

###Вспомогательные структуры и функции

In [ ]:
# Храним историю и профиль в памяти
PROFILE_FIELDS: List[Tuple[str, str]] = [
    ("С чем связан ваш интерес к нейронным сетям", "С чем связан ваш интерес к нейронным сетям?"),
    ("Текущая профессия", "Спасибо! Уточните, пожалуйста, какая у вас сейчас профессия?"),
    ("Текущее место работы", "Спасибо! А где вы сейчас работаете?"),
    ("Что нравится и не нравится в текущей работе", "Что вам нравится и не нравится в вашей текущей работе?"),
    ("Есть ли опыт программирования", "Есть ли у вас опыт программирования?"),
    ("Какая цель изучения AI", "Какова ваша основная цель изучения AI?")
]

user_profile: Dict[str, Optional[str]] = {key: None for key, _ in PROFILE_FIELDS}
presentation_text: str = ""
profile_complete: bool = False

def next_empty_field() -> Tuple[Optional[str], Optional[str]]:
    """
    Находит первое незаполненное поле в профиле пользователя.

    Проверяет `user_profile` в порядке, определённом в `PROFILE_FIELDS`.
    Возвращает ключ и соответствующий ему вопрос для первого поля,
    значение которого равно `None`.

    Returns:
        Tuple[Optional[str], Optional[str]]: Кортеж (ключ, вопрос) для следующего
        незаполненного поля. Если все поля уже заполнены, возвращает (None, None).
    """
    for key, question in PROFILE_FIELDS:
        if user_profile[key] is None:
            return key, question
    return None, None

def is_profile_complete() -> bool:
    """
    Проверяет, все ли поля в профиле пользователя заполнены.

    Итерируется по ключам из `PROFILE_FIELDS` и проверяет, что для каждого
    ключа в `user_profile` установлено значение (не `None`).

    Returns:
        bool: `True`, если все поля профиля заполнены, иначе `False`.
    """
    return all(user_profile.get(key) is not None for key, _ in PROFILE_FIELDS)

###Отправление запроса к OpenAI и возврат ответа

In [ ]:
def get_ai_response(current_history):
    """
    Отправляет запрос к OpenAI с текущей историей диалога и системным промптом.
    Возвращает ответ в виде JSON-строки.
    """
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + current_history
    try:
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            response_format={"type": "json_object"},
            temperature=0.2
        )
        return completion.choices[0].message.content
    except Exception as e:
        error_message = {"error": f"Произошла ошибка API: {e}"}
        return json.dumps(error_message, ensure_ascii=False)

###Сбор истории сообщений и новое сообщение пользователя

In [ ]:
def _prepare_messages_for_ai(request: ChatRequest) -> List[Dict[str, str]]:
    """Собирает историю сообщений и новое сообщение пользователя для отправки в модель."""
    history = [{"role": m.role, "content": m.content} for m in request.history]
    history.append({"role": "user", "content": request.message})
    return history

###Парсинг ответа от модели

In [ ]:
def _parse_ai_response(raw_response: str) -> Dict:
    """Парсит JSON-ответ от модели, обрабатывая возможные ошибки."""
    try:
        parsed = json.loads(raw_response)
        return {
            "thoughts": parsed.get("thoughts"),
            "answer": parsed.get("answer", "Извините, произошла ошибка."),
            "system_commands": parsed.get("system_commands", {}),
        }
    except json.JSONDecodeError:
        # Если модель вернула не JSON, считаем весь ответ текстом
        return {
            "thoughts": "AI response was not valid JSON.",
            "answer": raw_response,
            "system_commands": {},
        }

###Обновление профиля пользователя

In [ ]:
def _process_ai_commands(system_commands: Dict, current_profile: Dict, llm_answer: str) -> Dict:
    """Обновляет профиль пользователя и флаги на основе команд от AI."""
    global presentation_text

    # 1. Обновляем глобальный профиль из данных ответа
    parsed_user_profile = system_commands.get('user_profile', {})
    current_profile.update(parsed_user_profile)

    # 2. Проверяем, завершено ли заполнение профиля
    if is_profile_complete():
        # Используем ответ модели как текст презентации, если профиль полон
        presentation_text = llm_answer
        system_commands['showForm'] = True
        system_commands['determined_branch'] = system_commands.get('determined_branch', 'основной сценарий')
    else:
        system_commands['showForm'] = False

    # 3. Убеждаемся, что в ответе будет самый актуальный профиль
    system_commands['user_profile'] = current_profile.copy()

    return system_commands

###Эндпоинт чата

In [ ]:
@app.post("/chat", response_model=ChatResponse)
async def chat(req: ChatRequest):
    """Основной эндпоинт чата, управляющий диалогом с AI."""
    global user_profile # Используем глобальный профиль для сессии

    # Шаг 1: Подготовка сообщений для AI
    messages_to_send = _prepare_messages_for_ai(req)

    # Шаг 2: Получение и отладка ответа от AI
    raw_ai_response = get_ai_response(messages_to_send)
    print("DEBUG LLM reply:", raw_ai_response)

    # Шаг 3: Парсинг ответа
    parsed_response = _parse_ai_response(raw_ai_response)

    # Шаг 4: Обработка системных команд и обновление состояния
    final_commands = _process_ai_commands(
        system_commands=parsed_response["system_commands"],
        current_profile=user_profile,
        llm_answer=parsed_response["answer"]
    )

    # Шаг 5: Формирование и отправка ответа клиенту
    return ChatResponse(
        thoughts=parsed_response["thoughts"],
        answer=parsed_response["answer"],
        system_commands=SystemCommands(
            determined_branch=final_commands.get('determined_branch'),
            showForm=final_commands.get('showForm'),
            user_profile=final_commands.get('user_profile', {})
        )
    )

## Запуск сервера
Полученную ссылку вставьте в переменную BACKEND_URL в конструкторе Stackblitz для запуска фронтенда

[Ссылка на интерфейс (файл App.js в песочнице stackblitz)](https://stackblitz.com/edit/vitejs-vite-qj3brueb?file=src%2FApp.tsx)

In [ ]:
if __name__ == "__main__":
    nest_asyncio.apply()
    public_url = ngrok.connect(8000).public_url
    print("Ваш FastAPI backend доступен по адресу:", public_url)

    # Конфигурация и запуск Uvicorn
    config = uvicorn.Config(app=app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    server.run()

Ваш FastAPI backend доступен по адресу: https://1324017cc394.ngrok-free.app


INFO:     Started server process [1309]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     194.9.224.212:0 - "OPTIONS /chat HTTP/1.1" 200 OK
INFO:     194.9.224.212:0 - "OPTIONS /chat HTTP/1.1" 200 OK
DEBUG LLM reply: {
  "thoughts": "Начинаем диалог с пользователем, активная ветка - основной сценарий, активный блок - Блок 1: Квалификация и определение цели, активный агент - агент-квалификации. Первый вопрос из профайла.",
  "answer": "Здравствуйте! С чем связан ваш интерес к нейронным сетям?",
  "system_commands": {
    "determined_branch": "основной сценарий",
    "showForm": false,
    "user_profile": {
      "С чем связан интерес к нейронным сетям": null,
      "Текущая профессия": null,
      "Текущее место работы": null,
      "Что нравится и не нравится в текущей работе": null,
      "Есть ли опыт программирования": null,
      "Какая цель изучения AI": null
    }
  }
}
INFO:     194.9.224.212:0 - "POST /chat HTTP/1.1" 200 OK
DEBUG LLM reply: {
  "thoughts": "Начинаем диалог с пользователем, активная ветка - основной сценарий, активный блок - Блок 1: Квалифи

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1309]


KeyboardInterrupt: 